In [ ]:
import numpy as np
import math
from opentrons import protocol_api
metadata = {'apiLevel':'2.8',
           'protocolName' : 'Mag-Bind® Ultra-Pure Plasmid 96 Kit with Lysate Clearance via Magnetic Beads',
           'author': 'Team 5'}


#note api 2.0 does not support height from base as an argument for magnetic_module.engage(). pass argument height = 20.0 if on api 2.0
#engagement height needs to be determined by user for their specific labware

def run(protocol: protocol_api.ProtocolContext):
    #labware

    reservoir = protocol.load_labware('nest_12_reservoir_15ml', 1)
    plate = protocol.load_labware('nest_96_wellplate_2ml_deep', 5)
    plate_2 = protocol.load_labware('nest_96_wellplate_2ml_deep', 6)
    plate_3 = protocol.load_labware('nest_96_wellplate_200ul_flat', 3)
    waste_reservoir = protocol.load_labware('agilent_1_reservoir_290ml', 2)
    mag_mod = protocol.load_module('magnetic module', 4)
    mag_plate = mag_mod.load_labware('nest_96_wellplate_2ml_deep')

    tips = protocol.load_labware('opentrons_96_tiprack_300ul', 10)
    tips2 = protocol.load_labware('opentrons_96_tiprack_300ul', 11)
    p300 = protocol.load_instrument('p300_multi_gen2', 'left', tip_racks=[tips, tips2])

    sol1 = reservoir['A1']
    sol2 = reservoir['A2']
    sol3 = reservoir['A3']
    ETR = reservoir['A4']
    mag = reservoir['A5']
    waste = waste_reservoir['A1']
    VHB = reservoir['A6']
    SPM = reservoir['A7']
    EB = reservoir['A8']

    # locations

    first_plate = plate.wells_by_name()
    second_plate = plate_2.wells_by_name()
    magplate = mag_plate.wells_by_name()


def custom_mix(mix, volume):

    if mix == 1:
        p300.mix(30, 0.4*volume, rate = 0.2) # gentle mix
    elif mix == 2:
        p300.mix(10, 0.4*volume, rate = 1.0) #normal mix
    elif mix == 3:
        p300.mix(15, 0.4*volume, rate = 1.5) #vigorous mix to resuspend
    else:
    
        pass
    p300.blow_out()

def tipFate(new_tip):
    if new_tip==True:
        p300.drop_tip()
    else:
        p300.return_tip()



def transfer(source, destination, volume, tiprack, tip, aPlate, aRes, mix):

    #source = aspirate location
    #destination = dispense location
    #volume = volume to transfer
    #tiprack = specific tiprack to load from. There's a choice of two in this protocol
    #tip decides tip fate, False to return tip, True to trash tip
    #aPlate specifies the source labware type, True implies source is a 96 well plate, False implies a Reservoir
    #aRes specifies the destination labware type, True implies a reservoir, False implies a 96 well plate

    new = volume//250 
    #volume transfers are done in blocks up to 250 to accommodate volume transfers greater than 300 uL
    #new specifies number of 250 ul transfers

    excess = int(volume - (250*new))
    #remaining volume to transfer afer new number of transfers

    p300.starting_tip = tiprack.rows()[0][0]
    #aligns pipette to A1 of tiprack

    if aPlate == True:
        sourceA = source.rows()[0]
        sourceB = source.wells_by_name()
    if a > 0 and a!= 12:    
        if volume <= 300:
            for i in range(a):
                p300.pick_up_tip()
                p300.aspirate(volume, sourceA[i] if aPlate == True else source)
                p300.dispense(volume, destination.rows()[0][i] if aRes == False else destination)
                p300.touch_tip() if aRes == False else None
                p300.blow_out(destination.rows()[0][i]) if aRes == False else p300.blow_out(waste)
                custom_mix(mix, volume)

                tipFate(tip)
        else:
            for i in range(a):
                p300.pick_up_tip()
                for f in range(new):

                    p300.aspirate(250, sourceA[i] if aPlate == True else source)
                    p300.dispense(250, destination.rows()[0][i] if aRes == False else destination)
                    p300.touch_tip() if aRes == False else None
                    p300.blow_out(destination.rows()[0][i]) if aRes == False else p300.blow_out(waste)

                if excess != 0:
                    p300.aspirate(excess, sourceA[i] if aPlate == True else source)
                    p300.dispense(excess, destination.rows()[0][i] if aRes == False else destination)
                    p300.touch_tip() if aRes == False else None
                    p300.blow_out(destination.rows()[0][i]) if aRes == False else p300.blow_out(waste)
                    custom_mix(mix, volume)
                tipFate(tip)
        if b ==0:
            p300.reset_tipracks()
        if b != 0:
            if volume <=300:
                p300.pick_up_tip(tiprack.wells_by_name()[dict[b] + str(a+1)])
                p300.aspirate(volume, sourceB[dict[b] + str(a+1)] if aPlate == True else source)
                p300.dispense(volume, destination.wells_by_name()[dict[b] + str(a+1)] if aRes == False else destination)
                p300.touch_tip() if aRes == False else None
                p300.blow_out(destination.wells_by_name()[dict[b] + str(a+1)]) if aRes == False else p300.blow_out(waste)
                custom_mix(mix, volume)
                tipFate(tip)
            else:
                p300.pick_up_tip(tiprack.wells_by_name()[dict[b] + str(a+1)])
                for i in range(new):

                    p300.aspirate(250, sourceB[dict[b] + str(a+1)] if aPlate == True else source)
                    p300.dispense(250, destination.wells_by_name()[dict[b] + str(a+1)] if aRes == False else destination)
                    p300.touch_tip() if aRes == False else None
                    p300.blow_out(destination.wells_by_name()[dict[b] + str(a+1)]) if aRes == False else p300.blow_out(waste)

                if excess != 0:
                    p300.aspirate(excess, sourceB[dict[b] + str(a+1)] if aPlate == True else source)
                    p300.dispense(excess, destination.wells_by_name()[dict[b] + str(a+1)] if aRes == False else destination)
                    p300.touch_tip() if aRes == False else None
                    p300.blow_out(destination.wells_by_name()[dict[b] + str(a+1)]) if aRes == False else p300.blow_out(waste)
                    custom_mix(mix, volume)
                tipFate(tip)
        p300.reset_tipracks()
    if a == 12:
        if volume <=300:
            for i in range(12):
                p300.pick_up_tip()
                p300.aspirate(volume, sourceA[i] if aPlate == True else source)
                p300.dispense(volume, destination.rows()[0][i] if aRes == False else destination)
                p300.touch_tip() if aRes == False else None
                p300.blow_out(destination.rows()[0][i]) if aRes == False else p300.blow_out(waste)
                custom_mix(mix, volume)
                tipFate(tip)
        else:
            for i in range (12):
                p300.pick_up_tip()
                for f in range(new):
                    p300.aspirate(250, sourceA[i] if aPlate == True else source)
                    p300.dispense(250, destination.rows()[0][i] if aRes == False else destination)
                    p300.touch_tip() if aRes == False else None
                    p300.blow_out(destination.rows()[0][i]) if aRes == False else p300.blow_out(waste)
                    custom_mix(mix, volume)

                if excess != 0:
                    p300.aspirate(excess, sourceA[i] if aPlate == True else source)
                    p300.dispense(excess, destination.rows()[0][i] if aRes == False else destination)
                    p300.touch_tip() if aRes == False else None
                    p300.blow_out(destination.rows()[0][i]) if aRes == False else p300.blow_out(waste)
                    custom_mix(mix, volume)
                tipFate(tip)
        p300.reset_tipracks()
    if b != 0 and a==0:
        if volume <= 300:
            p300.pick_up_tip(tiprack.wells_by_name()[dict[b] + str(a+1)])
            p300.aspirate(volume, sourceB[dict[b] + str(a+1)] if aPlate == True else source)
            p300.dispense(volume, destination.wells_by_name()[dict[b] + str(a+1)] if aRes == False else destination)
            p300.touch_tip() if aRes == False else None
            p300.blow_out(destination.wells_by_name()[dict[b] + str(a+1)]) if aRes == False else p300.blow_out(waste)
            custom_mix(mix, volume)
            tipFate(tip)
        else:
            p300.pick_up_tip(tiprack.wells_by_name()[dict[b] + str(a+1)])
            for i in range(new):
                p300.aspirate(250, sourceB[dict[b] + str(a+1)] if aPlate == True else source)
                p300.dispense(250, destination.wells_by_name()[dict[b] + str(a+1)] if aRes == False else destination)
                p300.touch_tip() if aRes == False else None
                p300.blow_out(destination.wells_by_name()[dict[b] + str(a+1)]) if aRes == False else p300.blow_out(waste)
                custom_mix(mix, volume)

            if excess != 0:
                p300.aspirate(excess, source[dict[b] + str(a+1)] if aPlate == True else source)
                p300.dispense(excess, destination.wells_by_name()[dict[b] + str(a+1)] if aRes == False else destination)
                p300.touch_tip() if aRes == False else None
                p300.blow_out(destination.wells_by_name()[dict[b] + str(a+1)]) if aRes == False else p300.blow_out(waste)
                custom_mix(mix, volume)
            tipFate(tip)
        p300.reset_tipracks()

    sample_number=20

    dict = {1:'H', 2:'G', 3:'F', 4:'E', 5:'D', 6:'C', 7:'B', 8:'C'}

    a = sample_number//8
    b = sample_number%8

    #######################################################Let's start !!! ############################################
    # 3. remove supernatant:

    transfer(plate, waste, 1000, tips, tip=False, aPlate = True, aRes = True, mix = 0)

    #4. add solution I/RNase A

    transfer(sol1, plate, 250, tips2, tip=False, aPlate = False, aRes = False, mix = 0)

    #5. add solution II

    transfer(sol2, plate, 250, tips2, tip=False, aPlate = False, aRes = False, mix = 0)
    transfer(plate, plate, 0, tips, tip=False, aPlate = True, aRes = False, mix = 2)

    #6. add N3 buffer and LC beads
    transfer(sol3, plate, 155, tips2, tip=False, aPlate = False, aRes=False, mix = 0)
    transfer(plate, plate, 0, tips, tip=False, aPlate = True, aRes=False, mix = 1)

    #7. transfer clear lysate to magnetic module plate and engage magnet
    transfer(plate, mag_plate, 700, tips, tip=False, aPlate=True, aRes=False, mix = 0)
    mag_mod.engage(height = 20.0)

    # 8. transfer supernatant to a clean 96 well plate
    transfer(mag_plate, plate_2, 500, tips, tip=False, aPlate=True, aRes=False, mix = 0)
    mag_mod.disengage()

    # 9. add ETR and Mag particle
    transfer(ETR, plate_2, 500, tips2, tip=False, aPlate=False, aRes= False, mix = 0)
    transfer(mag, plate_2, 20, tips3, tip=True, aPlate=False, aRes=False, mix = 2) # might need to be tips2, was tips

    # 10. wait for 5 minutes
    protocol.delay(seconds = 300)

    # 11. return lysate to magnetic plate
    transfer(plate_2, mag_plate, 950, tips, tip=False, aPlate=True, aRes=False, mix = 0)
    mag_mod.engage(height = 20.0)
    protocol.delay(seconds = 300)

    # 12 aspirate and discard supernatant
    transfer(mag_plate, waste, 900, tips, tip=False, aPlate=True, aRes=True, mix = 0)

    # 13 remove magentic module
    mag_mod.disengage()

    # 14. add ETR wash buffer and resuspend
    transfer(ETR, mag_plate, 500, tips2, tip=False, aPlate=False, aRes=False, mix = 0) #should this be mag plate? was orignally dest plate 2
    transfer(mag_plate, mag_plate, 0, tips, tip=False, aPlate=True, aRes=False, mix = 2)
    mag_mod.engage(height = 20.0)
    protocol.delay(seconds = 300)

    # 15. remove supernatant
    transfer(mag_plate, waste, 500, tips, tip=False, aPlate=True, aRes=True, mix = 0)
    mag_mod.disengage()

    for q in range(2):
        # 16. add VHB buffer and resuspend
        transfer(VHB, mag_plate, 700, tips2, tip=False, aPlate=False, aRes=False, mix = 0 )
        transfer(mag_plate, mag_plate, 0, tips, tip=False, aPlate=True, aRes=False, mix = 2 )
        mag_mod.engage(height = 20.0)
        protocol.delay(seconds = 300)

        #17. discard supernatant
        transfer(mag_plate, waste, 650, tips, tip = False, aPlate = True, aRes = True, mix = 0)
        mag_mod.disengage()

    # 18. resuspend beads in SPM buffer
    transfer(SPM, mag_plate, 700, tips2, tip = False, aPlate = False, aRes = False, mix = 0)
    transfer(mag_plate, mag_plate, 0, tips, tip = False, aPlate = True, aRes = False, mix = 2)
    mag_mod.engage(height = 20.0)
    protocol.delay(seconds = 300)

    #19. incubate for 10 minutes on the magnet
    transfer(mag_plate, waste, 700, tips, tip = False, aPlate = True, aRes = True, mix = 0)

    protocol.delay(seconds = 600)

    mag_mod.disengage()

    #20. add elution buffer and mix
    transfer(EB, mag_plate, 100, tips2, tip = True, aPlate = False, aRes = False, mix = 0)
    transfer(mag_plate, mag_plate, 0, tips, tip = True, aPlate = True, aRes = False, mix = 2)

    #21. engange magnet and clear supernatant
    mag_mod.engage(height = 20.0)

    transfer(mag_plate, plate_3, 100, tips4, tip = True, aPlate = True, aRes = False, mix = 0) #gives out of bounds error but not a big deal
